In [395]:
# BMED Free Running Simulator
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import warnings
warnings.filterwarnings('ignore', category=RuntimeWarning)

print("🎯 BMED 1stage optimizer")
print("="*60)

🎯 BMED 1stage optimizer


In [396]:
# Model Classes (same as reference)

class LayerNormLSTM(nn.Module):
    """LSTM layer with layer normalization applied to gates"""
    def __init__(self, input_node, hidden_node):
        super().__init__()
        self.input_node = input_node
        self.hidden_node = hidden_node

        self.w_i = nn.Linear(input_node, 4 * hidden_node, bias=False)
        self.w_h = nn.Linear(hidden_node, 4 * hidden_node, bias=False)

        self.ln_i = nn.LayerNorm(hidden_node)
        self.ln_h = nn.LayerNorm(hidden_node)
        self.ln_g = nn.LayerNorm(hidden_node)
        self.ln_o = nn.LayerNorm(hidden_node)

        self.ln_c = nn.LayerNorm(hidden_node)

    def forward(self, input, hidden):
        h_prev, c_prev = hidden

        gi = self.w_i(input)
        gh = self.w_h(h_prev)
        i_i, i_f, i_g, i_o = gi.chunk(4, dim=-1)
        h_i, h_f, h_g, h_o = gh.chunk(4, dim=-1)

        i_g = torch.sigmoid(self.ln_i(i_i + h_i))
        f_g = torch.sigmoid(self.ln_h(i_f + h_f))
        g_g = torch.tanh(self.ln_g(i_g + h_g))
        o_g = torch.sigmoid(self.ln_o(i_o + h_o))

        c_new = f_g * c_prev + i_g * g_g
        c_new = self.ln_c(c_new)

        h_new = o_g * torch.tanh(c_new)

        return h_new, c_new

class StateExtr(nn.Module):
    """State Extractor using LayerNorm LSTM"""
    def __init__(self, input_node, hidden_node, n_layer, dropout):
        super().__init__()
        self.hidden_node = hidden_node
        self.n_layer = n_layer
        self.input_node = input_node

        self.lstm_cells = nn.ModuleList()
        self.lstm_cells.append(LayerNormLSTM(input_node, hidden_node))

        for _ in range(n_layer - 1):
            self.lstm_cells.append(LayerNormLSTM(hidden_node, hidden_node))

        self.dropout = nn.Dropout(dropout)
        self.final_layer_norm = nn.LayerNorm(hidden_node)
        self.final_dropout = nn.Dropout(dropout)

    def forward(self, x, seq_len):
        batch_size, max_len, input_node = x.size()
        device = x.device

        h_states = []
        c_states = []
        for _ in range(self.n_layer):
            h_states.append(torch.zeros(batch_size, self.hidden_node, device=device))
            c_states.append(torch.zeros(batch_size, self.hidden_node, device=device))
        
        outputs = []
        for t in range(max_len):
            x_t = x[:, t, :]

            layer_input = x_t
            for layer_idx, lstm_cell in enumerate(self.lstm_cells):
                h_new, c_new = lstm_cell(layer_input, (h_states[layer_idx], c_states[layer_idx]))

                h_states[layer_idx] = h_new
                c_states[layer_idx] = c_new

                if layer_idx < len(self.lstm_cells) - 1:
                    layer_input = self.dropout(h_new)
                else:
                    layer_input = h_new

            outputs.append(layer_input)
        
        output_tensor = torch.stack(outputs, dim=1)
        seq_len_cpu = seq_len.detach().cpu().long()
        mask = torch.arange(max_len, device='cpu')[None, :] < seq_len_cpu[:, None]
        mask = mask.float().to(device).unsqueeze(-1)

        masked_output = output_tensor * mask
        normalized = self.final_layer_norm(masked_output)
        return self.final_dropout(normalized)

class PhysicalChangeDecoder(nn.Module):
    """Physical Change Decoder"""
    def __init__(self, input_node, output_node, n_layer, hidden_node, dropout):
        super().__init__()

        self.layers = nn.ModuleList()

        self.layers.append(nn.Linear(input_node, hidden_node))
        self.layers.append(nn.LayerNorm(hidden_node))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Dropout(dropout))

        for i in range(n_layer - 1):
            self.layers.append(nn.Linear(hidden_node, hidden_node))
            self.layers.append(nn.LayerNorm(hidden_node))
            self.layers.append(nn.ReLU())
            self.layers.append(nn.Dropout(dropout))

        self.layers.append(nn.Linear(hidden_node, output_node))
    
    def forward(self, hidden_states):
        x = hidden_states
        for layer in self.layers:
            x = layer(x)
        return x

class CurrentPredictor(nn.Module):
    """Current Predictor"""
    def __init__(self, input_node, hidden_node, n_layer, dropout):
        super().__init__()
        
        self.layers = nn.ModuleList()
        
        self.layers.append(nn.Linear(input_node, hidden_node))
        self.layers.append(nn.LayerNorm(hidden_node))
        self.layers.append(nn.ReLU())
        self.layers.append(nn.Dropout(dropout))
        
        for i in range(n_layer - 1):
            self.layers.append(nn.Linear(hidden_node, hidden_node))
            self.layers.append(nn.LayerNorm(hidden_node))
            self.layers.append(nn.ReLU())
            self.layers.append(nn.Dropout(dropout))
        
        self.layers.append(nn.Linear(hidden_node, 1))
    
    def forward(self, new_state):
        x = new_state
        for layer in self.layers:
            x = layer(x)
        return x

print("✅ 모델 클래스 정의 완료")

✅ 모델 클래스 정의 완료


In [397]:
class PhysicsConstraintLayer(nn.Module):
    """Physics Constraint Layer with Current Prediction and Direct ReLU Constraints"""
    def __init__(self, range_mm, current_predictor, eps=1e-2):
        super().__init__()
        self.sps = eps
        self.current_predictor = current_predictor
        self.register_buffer('range_mm_tensor', self._convert_range_to_tensor(range_mm))

    def _convert_range_to_tensor(self, range_mm):
        feature_names = ['V','E','VF','VA','VB','CFLA','CALA','CFK','CBK','I']
        ranges = torch.zeros(len(feature_names),2)

        for i, name in enumerate(feature_names):
            if name in range_mm:
                ranges[i, 0] = range_mm[name]['min']
                ranges[i, 1] = range_mm[name]['max']
        
        return ranges
    
    def normalize(self, data, feature_idx):
        min_val = self.range_mm_tensor[feature_idx, 0]
        max_val = self.range_mm_tensor[feature_idx, 1]
        return (data - min_val) / (max_val - min_val)

    def denormalize(self, data, feature_idx):
        min_val = self.range_mm_tensor[feature_idx, 0]
        max_val = self.range_mm_tensor[feature_idx, 1]
        return data * (max_val - min_val) + min_val

    def forward(self, physical_changes, current_state):
        V_idx, E_idx, VF_idx, VA_idx, VB_idx = 0, 1, 2, 3, 4
        CFLA_idx, CALA_idx, CFK_idx, CBK_idx, I_idx = 5, 6, 7, 8, 9

        VF = self.denormalize(current_state[..., 2:3], VF_idx)
        VA = self.denormalize(current_state[..., 3:4], VA_idx)
        VB = self.denormalize(current_state[..., 4:5], VB_idx)
        CFLA = self.denormalize(current_state[..., 5:6], CFLA_idx)
        CALA = self.denormalize(current_state[..., 6:7], CALA_idx)
        CFK = self.denormalize(current_state[..., 7:8], CFK_idx)
        CBK = self.denormalize(current_state[..., 8:9], CBK_idx)

        dVA = physical_changes[..., 0:1]
        dVB = physical_changes[..., 1:2]
        rratio = physical_changes[..., 2:3]
        dNBK = physical_changes[..., 3:4]

        ratio = torch.sigmoid(rratio)
        dNALA = ratio * dNBK
        
        NFLA = CFLA * VF
        NALA = CALA * VA
        NFK = CFK * VF
        NBK = CBK * VB

        # tensor 비교를 torch.where로 변경
        condition1 = VF < dVA + dVB
        dVA = torch.where(condition1, torch.zeros_like(dVA), dVA)
        dVB = torch.where(condition1, torch.zeros_like(dVB), dVB)
        
        condition2 = NFLA < dNALA
        dNALA = torch.where(condition2, torch.zeros_like(dNALA), dNALA)
        
        condition3 = NFK < dNBK
        dNBK = torch.where(condition3, torch.zeros_like(dNBK), dNBK)

        nVF = VF - dVA - dVB
        nVA = VA + dVA
        nVB = VB + dVB

        nVF = torch.clamp(nVF, min=self.sps)
        nVA = torch.clamp(nVA, min=self.sps)
        nVB = torch.clamp(nVB, min=self.sps)
        
        nNFLA = NFLA - dNALA
        nNALA = NALA + dNALA
        nNFK = NFK - dNBK
        nNBK = NBK + dNBK

        nCFLA = nNFLA / nVF
        nCALA = nNALA / nVA
        nCFK = nNFK / nVF
        nCBK = nNBK / nVB

        V = current_state[..., 0:1]
        E = current_state[..., 1:2]
        nVF_norm = self.normalize(nVF, VF_idx)
        nVA_norm = self.normalize(nVA, VA_idx)
        nVB_norm = self.normalize(nVB, VB_idx)
        nCFLA_norm = self.normalize(nCFLA, CFLA_idx)
        nCALA_norm = self.normalize(nCALA, CALA_idx)
        nCFK_norm = self.normalize(nCFK, CFK_idx)
        nCBK_norm = self.normalize(nCBK, CBK_idx)

        temp_state = torch.cat([
            V, E, nVF_norm, nVA_norm, nVB_norm, nCFLA_norm, nCALA_norm, nCFK_norm, nCBK_norm
        ], dim=-1)
        
        nI_pred_norm = self.current_predictor(temp_state)
        
        nI_real = self.denormalize(nI_pred_norm, I_idx)
        nI_real = torch.clamp(nI_real, min=0.0)
        nI_norm = self.normalize(nI_real, I_idx)

        next_state = torch.cat([
            V, E, nVF_norm, nVA_norm, nVB_norm, nCFLA_norm, nCALA_norm, nCFK_norm, nCBK_norm, nI_norm
        ], dim=-1)
        
        return next_state

class BMEDFreeRunningModel(nn.Module):
    """BMED Free Running Model for Simulation"""
    def __init__(self, state_extr_params, decoder_params, current_predictor_params, range_mm):
        super().__init__()
        self.state_extr = StateExtr(**state_extr_params)
        self.physical_decoder = PhysicalChangeDecoder(**decoder_params)
        self.current_predictor = CurrentPredictor(**current_predictor_params)
        self.physics_constraint = PhysicsConstraintLayer(range_mm, self.current_predictor)
        
        self._hidden_states = None
        self._cell_states = None

    def _reset_hidden_states(self, batch_size, device):
        self._hidden_states = []
        self._cell_states = []
        for _ in range(self.state_extr.n_layer):
            self._hidden_states.append(torch.zeros(batch_size, self.state_extr.hidden_node, device=device))
            self._cell_states.append(torch.zeros(batch_size, self.state_extr.hidden_node, device=device))

    def free_running_forward(self, initial_state, target_length):
        batch_size = initial_state.size(0)
        feature_size = initial_state.size(1)
        device = initial_state.device
        
        self._reset_hidden_states(batch_size, device)
        
        predictions = torch.zeros(batch_size, target_length, feature_size, device=device)
        current_state = initial_state.clone()
        
        for t in range(target_length):
            predictions[:, t, :] = current_state
            
            if t < target_length - 1:
                lstm_input = current_state[:, :-1]
                hidden_output = self._forward_lstm_single_step(lstm_input)
                physical_changes = self.physical_decoder(hidden_output.unsqueeze(1))
                current_state_expanded = current_state.unsqueeze(1)
                next_state = self.physics_constraint(physical_changes, current_state_expanded)
                current_state = next_state.squeeze(1)
        
        return predictions

    def _forward_lstm_single_step(self, x_t):
        layer_input = x_t
        
        for layer_idx, lstm_cell in enumerate(self.state_extr.lstm_cells):
            h_new, c_new = lstm_cell(layer_input, 
                                   (self._hidden_states[layer_idx], self._cell_states[layer_idx]))
            
            self._hidden_states[layer_idx] = h_new
            self._cell_states[layer_idx] = c_new
            
            if layer_idx < len(self.state_extr.lstm_cells) - 1:
                layer_input = self.state_extr.dropout(h_new)
            else:
                layer_input = h_new
        
        normalized = self.state_extr.final_layer_norm(layer_input)
        return self.state_extr.final_dropout(normalized)

    def forward(self, initial_state, target_length):
        return self.free_running_forward(initial_state, target_length)

print("✅ ReLU 기반 직접 제약이 적용된 Physics Constraint Layer 정의 완료")

✅ ReLU 기반 직접 제약이 적용된 Physics Constraint Layer 정의 완료


In [398]:
# 학습된 모델 로드
model_path = "BMED_FR_250909.pth"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f"📥 모델 파일 로드 중: {model_path}")
print(f"🔧 사용 중인 장치: {device}")

try:
    checkpoint = torch.load(model_path, map_location=device, weights_only=False)
    
    # 모델 설정 정보 확인
    model_config = checkpoint['model_config']
    state_extr_params = model_config['state_extr_params']
    decoder_params = model_config['decoder_params']
    current_predictor_params = model_config['current_predictor_params']
    model_range_mm = model_config['range_mm']
    
    # 모델 생성 및 가중치 로드
    simulator_model = BMEDFreeRunningModel(
        state_extr_params=state_extr_params,
        decoder_params=decoder_params, 
        current_predictor_params=current_predictor_params,
        range_mm=model_range_mm
    ).to(device)
    
    simulator_model.load_state_dict(checkpoint['model_state_dict'])
    simulator_model.eval()
    
    print("✅ 시뮬레이터 모델 준비 완료!")
    print(f"   - 모델 성능 (Best FR Loss): {checkpoint.get('best_fr_loss', 'Unknown'):.6f}")
    
except FileNotFoundError:
    print(f"❌ 모델 파일을 찾을 수 없습니다: {model_path}")
    print("   현재 디렉토리에 BMED_FR_250817.pth 파일이 있는지 확인하세요.")
    simulator_model = None
    model_range_mm = None

📥 모델 파일 로드 중: BMED_FR_250909.pth
🔧 사용 중인 장치: cuda
✅ 시뮬레이터 모델 준비 완료!
   - 모델 성능 (Best FR Loss): 0.000911


In [399]:
# 정규화/비정규화 함수
def normalize_inputs(inputs, range_mm):
    """입력값들을 모델에서 사용하는 범위로 정규화"""
    feature_names = ['V', 'E', 'VF', 'VA', 'VB', 'CFLA', 'CALA', 'CFK', 'CBK']
    normalized = []
    
    for i, (name, value) in enumerate(zip(feature_names, inputs)):
        if name in range_mm:
            min_val = range_mm[name]['min']
            max_val = range_mm[name]['max']
            norm_val = (value - min_val) / (max_val - min_val)
            normalized.append(norm_val)
        else:
            normalized.append(value)
    
    return normalized

def denormalize_outputs(outputs, range_mm):
    """출력값들을 실제 물리적 값으로 변환"""
    feature_names = ['V', 'E', 'VF', 'VA', 'VB', 'CFLA', 'CALA', 'CFK', 'CBK', 'I']
    denormalized = np.zeros_like(outputs)
    
    for i, name in enumerate(feature_names):
        if name in range_mm:
            min_val = range_mm[name]['min']
            max_val = range_mm[name]['max']
            denormalized[:, :, i] = outputs[:, :, i] * (max_val - min_val) + min_val
        else:
            denormalized[:, :, i] = outputs[:, :, i]
    
    return denormalized

In [400]:
# Simulation
def simulation(cond):
    V = cond[0]
    E = cond[1]
    VF = 1
    VA = 1
    VB = cond[2]
    CFLA = cond[3]
    CALA = 0
    CFK = CFLA*2
    CBK = 0
    simulation_time = 81  # 시뮬레이션할 시간 스텝 수

    input_values = [V, E, VF, VA, VB, CFLA, CALA, CFK, CBK]
    normalized_inputs = normalize_inputs(input_values, model_range_mm)

    # 초기 전류값 추정 (정규화된 0)
    initial_I_norm = 0.0

    # 초기 상태 텐서 생성 (10개 특성 모두 포함)
    initial_state_values = normalized_inputs + [initial_I_norm]
    initial_state_tensor = torch.tensor([initial_state_values]).float().to(device)

    # Free Running 시뮬레이션 실행
    with torch.no_grad():
        predictions = simulator_model(initial_state_tensor, simulation_time)

    # 결과를 numpy로 변환
    pred_normalized = predictions.cpu().numpy()  # [1, time_steps, 10]

    # 비정규화하여 실제 물리적 값으로 변환
    pred_physical = denormalize_outputs(pred_normalized, model_range_mm)

    # 시간 배열 생성
    time_steps = np.arange(simulation_time)

    # 결과 데이터 저장
    sim_res = {
        'time': time_steps,
        'V': pred_physical[0, :, 0],
        'E': pred_physical[0, :, 1], 
        'VF': pred_physical[0, :, 2],
        'VA': pred_physical[0, :, 3],
        'VB': pred_physical[0, :, 4],
        'CFLA': pred_physical[0, :, 5],
        'CALA': pred_physical[0, :, 6],
        'CFK': pred_physical[0, :, 7],
        'CBK': pred_physical[0, :, 8],
        'I': pred_physical[0, :, 9]
    }

    time = sim_res['time'] * 0.25 # hr
    VF = sim_res['VF'] # L
    VA = sim_res['VA'] # L
    VB = sim_res['VB'] # L
    CFLA = sim_res['CFLA'] # mol/L
    CALA = sim_res['CALA'] # mol/L
    CFK = sim_res['CFK'] # mol/L
    CBK = sim_res['CBK'] # mol/L
    I = sim_res['I'] # A
    NALA = CALA *VA # mol
    NBK = CBK *VB # mol
    MwLA=90.1 #g/mol
    Amem = 0.055 #m2
    flux_mol = np.zeros_like(NALA) # mol/hr
    flux_mol[0] = (NALA[1] - NALA[0]) / (time[1] - time[0])
    flux_mol[1:-1] = (NALA[2:] - NALA[:-2]) / (time[2:] - time[:-2])
    flux_mol[-1] = (NALA[-1] - NALA[-2]) / (time[-1] - time[-2])
    flux_mol = flux_mol/3600 # mol/s
    JLA = flux_mol*MwLA/Amem # g/m2/s
    CE = 96485*flux_mol/10/I*100 # %
    SEC = V*I/(MwLA*flux_mol*3600) # kwh/kg
    SEC = np.where(SEC < 0, 0, SEC)

    end_point = 0
    for i in range(len(flux_mol)-1):
        if i>4 and flux_mol[i+1] <= 1e-8:
            end_point = i
            break
        else:
            end_point = i

    time2 = time[:end_point]
    VF2 = VF[:end_point]
    VA2 = VA[:end_point]
    VB2 = VB[:end_point]
    CFLA2 = CFLA[:end_point]
    CALA2 = CALA[:end_point]
    CFK2 = CFK[:end_point]
    CBK2 = CBK[:end_point]
    I2 = I[:end_point]
    NALA2 = NALA[:end_point]
    NBK2 = NBK[:end_point]
    CE2 = np.insert(CE[1:end_point], 0, 0)
    JLA2 = JLA[:end_point]
    SEC2 = SEC[:end_point]
    res = {
        'time': time2,
        'VF': VF2,
        'VA': VA2,
        'VB': VB2,
        'CFLA': CFLA2,
        'CALA': CALA2,
        'CFK': CFK2,
        'CBK': CBK2,
        'I': I2,
        'CE': CE2,
        'NALA': NALA2,
        'NBK': NBK2,
        'JLA': JLA2,
        'SEC': SEC2,
    }
    return res

In [401]:
# 1kg LA 분리 기준 회수율 및 에너지 소모량 분석 85wt% 농축 기준
def performance(res, cond):
    V = cond[0]
    Amem = 0.055 #m2
    MwLA=90.1 #g/mol

    NLA0 = res['CFLA'][0] * res['VF'][0] # mol
    NK0 = res['CFK'][0] * res['VF'][0] # mol
    target_Cm = 0.85 # kg/L = kg/kg

    RR_LA = []
    RR_K = []
    E_BMED = []
    E_PUMP = []
    E_sensible = []
    E_vapor = []
    Total_E = []
    Total_Time = []

    for i in range(len(res['time'])-1):
        outLA = res['NALA'][i] # mol
        outK = res['NBK'][i] # mol

        target_VA = outLA*MwLA/1000/target_Cm # L = kg

        rr_LA = outLA/NLA0*100
        rr_K = outK/NK0*100

        dNLA = res['JLA'][i]*Amem*3600 # g/hr
        Fv_out = dNLA/MwLA/res['CALA'][i] # L/hr
        total_time = 1000/dNLA

        e_BMED = V*res['I'][i]*total_time/1000 # kWh/kg
        e_PUMP = 0.0047*total_time # kWh/kg
        e_sensible = res['VA'][i]*4.184*(100-20)*total_time/3600 # kWh/kg
        e_vapor = (res['VA'][i] - target_VA)*44/18*total_time/3600 # kWh/kg
        Total_e = e_BMED + e_PUMP + e_sensible + e_vapor # kWh/kg

        RR_LA.append(rr_LA)
        RR_K.append(rr_K)
        E_BMED.append(e_BMED)
        E_PUMP.append(e_PUMP)
        E_sensible.append(e_sensible)
        E_vapor.append(e_vapor)
        Total_E.append(Total_e)
        Total_Time.append(total_time)
    
    return RR_LA, RR_K, E_BMED, E_PUMP, E_sensible, E_vapor, Total_E, Total_Time



In [402]:
condition = [10, 0.25, 2, 0.5]
res = simulation(condition)
RR_LA, RR_K, E_BMED, E_PUMP, E_sensible, E_vapor, Total_E, Total_Time = performance(res,condition)
df = pd.DataFrame()
df['Voltage (V)'] = [condition[0]]*len(RR_LA)
df['Electrolyte (M)'] = [condition[1]]*len(RR_LA)
df['Feed Flow [L/hr]'] = [1]*len(RR_LA)
df['Acid Flow [L/hr]'] = [1]*len(RR_LA)
df['Base Flow [L/hr]'] = [condition[2]]*len(RR_LA)
df['Init LA (M)'] = [condition[3]]*len(RR_LA)
df['Init K (M)'] = [condition[3]*2]*len(RR_LA)
df['LA Recovery (%)'] = RR_LA
df['K Recovery (%)'] = RR_K
df['E Stack (kWh/kg)'] = E_BMED
df['E PUMP (kWh/kg)'] = E_PUMP
df['E Sensible (kWh/kg)'] = E_sensible
df['E Vapor (kWh/kg)'] = E_vapor
df['Total (kWh/kg)'] = Total_E
df['Operation Time (hr)'] = Total_Time
df = df[(df >= 0).all(axis=1)].reset_index(drop=True)
df

,Voltage (V),Electrolyte (M),Feed Flow [L/hr],Acid Flow [L/hr],Base Flow [L/hr],Init LA (M),Init K (M),LA Recovery (%),K Recovery (%),E Stack (kWh/kg),E PUMP (kWh/kg),E Sensible (kWh/kg),E Vapor (kWh/kg),Total (kWh/kg),Operation Time (hr)
0,10,0.25,1,1,2,0.5,1.0,0.169046,0.409186,0.954193,13.330544,255.857330,1.868341,272.010406,2836.286133
1,10,0.25,1,1,2,0.5,1.0,0.381437,0.918534,0.943983,11.629101,221.548126,1.617616,235.738831,2474.276855
2,10,0.25,1,1,2,0.5,1.0,0.617612,1.480953,1.012795,10.764801,203.568649,1.486143,216.832397,2290.383301
3,10,0.25,1,1,2,0.5,1.0,0.866019,2.067807,1.112331,10.209898,191.654541,1.398967,204.375748,2172.318848
4,10,0.25,1,1,2,0.5,1.0,1.128531,2.682391,1.200306,9.551015,177.973526,1.298907,190.023758,2032.130859
5,10,0.25,1,1,2,0.5,1.0,1.412184,3.339069,1.293796,8.950046,165.563965,1.208138,177.015945,1904.265137
6,10,0.25,1,1,2,0.5,1.0,1.711369,4.023028,1.415898,8.549619,157.018692,1.145580,168.129791,1819.067871
7,10,0.25,1,1,2,0.5,1.0,2.022319,4.724102,1.560674,8.264181,150.695831,1.099245,161.619934,1758.336426
8,10,0.25,1,1,2,0.5,1.0,2.342578,5.436004,1.667250,8.062675,145.985855,1.064682,156.780457,1715.462769
9,10,0.25,1,1,2,0.5,1.0,2.669304,6.152835,1.759169,7.959513,143.112198,1.043515,153.874405,1693.513428


In [403]:
# import numpy as np
# import pandas as pd
# from tqdm import tqdm

# # 조건 범위 설정
# voltages = np.arange(10, 36, 1)  # 10~35, 1단위
# electrolytes = np.arange(0.25, 1.01, 0.05)  # 0.25~1, 0.05단위
# base_flows = np.arange(2, 6.01, 0.5)  # 2~6, 0.5단위
# init_LAs = np.arange(0.5, 3.01, 0.1)  # 0.5~3, 0.1단위

# results = []

# for v in tqdm(voltages, desc="Voltage"):
#     for e in electrolytes:
#         for b in base_flows:
#             for la in init_LAs:
#                 print(f'Voltage: {v}, Electrolyte: {e}, Base Flow: {b}, Init LA: {la}')
#                 condition = [v, round(e, 2), round(b, 2), round(la, 2)]
#                 res = simulation(condition)
#                 RR_LA, RR_K, E_BMED, E_PUMP, E_sensible, E_vapor, Total_E, Total_Time = performance(res,condition)
#                 for i in range(len(RR_LA)):
#                     results.append({
#                         'Voltage (V)': v,
#                         'Electrolyte (M)': round(e, 2),
#                         'Feed Flow [L/hr]': 1,
#                         'Acid Flow [L/hr]': 1,
#                         'Base Flow [L/hr]': round(b, 2),
#                         'Init LA (M)': round(la, 2),
#                         'Init K (M)': round(la*2, 2),
#                         'LA Recovery (%)': RR_LA[i],
#                         'K Recovery (%)': RR_K[i],
#                         'E Stack (kWh/kg)': E_BMED[i],
#                         'E PUMP (kWh/kg)': E_PUMP[i],
#                         'E Sensible (kWh/kg)': E_sensible[i],
#                         'E Vapor (kWh/kg)': E_vapor[i],
#                         'Total (kWh/kg)': Total_E[i],
#                         'Operation Time (hr)': Total_Time[i]
#                     })
                

# df = pd.DataFrame(results)
# df = df[(df >= 0).all(axis=1)].reset_index(drop=True)
# df

In [404]:
df

,Voltage (V),Electrolyte (M),Feed Flow [L/hr],Acid Flow [L/hr],Base Flow [L/hr],Init LA (M),Init K (M),LA Recovery (%),K Recovery (%),E Stack (kWh/kg),E PUMP (kWh/kg),E Sensible (kWh/kg),E Vapor (kWh/kg),Total (kWh/kg),Operation Time (hr)
0,10,0.25,1,1,2,0.5,1.0,0.169046,0.409186,0.954193,13.330544,255.857330,1.868341,272.010406,2836.286133
1,10,0.25,1,1,2,0.5,1.0,0.381437,0.918534,0.943983,11.629101,221.548126,1.617616,235.738831,2474.276855
2,10,0.25,1,1,2,0.5,1.0,0.617612,1.480953,1.012795,10.764801,203.568649,1.486143,216.832397,2290.383301
3,10,0.25,1,1,2,0.5,1.0,0.866019,2.067807,1.112331,10.209898,191.654541,1.398967,204.375748,2172.318848
4,10,0.25,1,1,2,0.5,1.0,1.128531,2.682391,1.200306,9.551015,177.973526,1.298907,190.023758,2032.130859
5,10,0.25,1,1,2,0.5,1.0,1.412184,3.339069,1.293796,8.950046,165.563965,1.208138,177.015945,1904.265137
6,10,0.25,1,1,2,0.5,1.0,1.711369,4.023028,1.415898,8.549619,157.018692,1.145580,168.129791,1819.067871
7,10,0.25,1,1,2,0.5,1.0,2.022319,4.724102,1.560674,8.264181,150.695831,1.099245,161.619934,1758.336426
8,10,0.25,1,1,2,0.5,1.0,2.342578,5.436004,1.667250,8.062675,145.985855,1.064682,156.780457,1715.462769
9,10,0.25,1,1,2,0.5,1.0,2.669304,6.152835,1.759169,7.959513,143.112198,1.043515,153.874405,1693.513428


In [405]:
RR_LA

[np.float32(0.0),
 np.float32(-0.21188873),
 np.float32(-0.15344092),
 np.float32(-0.009876408),
 np.float32(0.16904555),
 np.float32(0.3814374),
 np.float32(0.6176122),
 np.float32(0.8660192),
 np.float32(1.1285307),
 np.float32(1.4121838),
 np.float32(1.7113686),
 np.float32(2.0223193),
 np.float32(2.3425777),
 np.float32(2.669304),
 np.float32(2.9979477),
 np.float32(3.323014),
 np.float32(3.6371608),
 np.float32(3.9371789),
 np.float32(4.2208443),
 np.float32(4.4871473),
 np.float32(4.7371116),
 np.float32(4.974238),
 np.float32(5.209576),
 np.float32(5.448859),
 np.float32(5.696504),
 np.float32(5.962212),
 np.float32(6.287679),
 np.float32(6.81366),
 np.float32(8.068373),
 np.float32(11.522874),
 np.float32(18.375805),
 np.float32(27.475851),
 np.float32(38.22382),
 np.float32(51.035397),
 np.float32(64.94035)]

In [406]:
Total_Time

[np.float32(-2619.011),
 np.float32(-7233.259),
 np.float32(5494.1094),
 np.float32(3441.626),
 np.float32(2836.2861),
 np.float32(2474.2769),
 np.float32(2290.3833),
 np.float32(2172.3188),
 np.float32(2032.1309),
 np.float32(1904.2651),
 np.float32(1819.0679),
 np.float32(1758.3364),
 np.float32(1715.4628),
 np.float32(1693.5134),
 np.float32(1697.8132),
 np.float32(1736.3192),
 np.float32(1807.1339),
 np.float32(1901.507),
 np.float32(2018.0746),
 np.float32(2149.8118),
 np.float32(2278.5876),
 np.float32(2349.1228),
 np.float32(2338.4492),
 np.float32(2279.348),
 np.float32(2162.0178),
 np.float32(1877.4105),
 np.float32(1303.5178),
 np.float32(623.2841),
 np.float32(235.68222),
 np.float32(107.67744),
 np.float32(69.57184),
 np.float32(55.918835),
 np.float32(47.109474),
 np.float32(41.542736),
 np.float32(39.362473)]

In [407]:
res['JLA']

array([-0.0019284 , -0.00069823,  0.00091926,  0.00146748,  0.00178068,
        0.0020412 ,  0.00220509,  0.00232494,  0.00248532,  0.00265221,
        0.00277642,  0.00287232,  0.00294411,  0.00298226,  0.00297471,
        0.00290874,  0.00279476,  0.00265605,  0.00250264,  0.00234928,
        0.00221651,  0.00214995,  0.00215977,  0.00221577,  0.00233601,
        0.00269014,  0.00387452,  0.00810305,  0.0214293 ,  0.04690402,
        0.07259411,  0.09031849,  0.10720783,  0.12157373,  0.12830761,
        0.1414937 ], dtype=float32)

In [408]:
total_time

np.float32(79.45821)

In [409]:
E_PUMP

[np.float32(-12.309352),
 np.float32(-33.996315),
 np.float32(25.822313),
 np.float32(16.175642),
 np.float32(13.330544),
 np.float32(11.629101),
 np.float32(10.764801),
 np.float32(10.209898),
 np.float32(9.551015),
 np.float32(8.950046),
 np.float32(8.549619),
 np.float32(8.264181),
 np.float32(8.0626745),
 np.float32(7.959513),
 np.float32(7.979722),
 np.float32(8.1607),
 np.float32(8.493529),
 np.float32(8.937082),
 np.float32(9.48495),
 np.float32(10.1041155),
 np.float32(10.709362),
 np.float32(11.040877),
 np.float32(10.990711),
 np.float32(10.712935),
 np.float32(10.161484),
 np.float32(8.82383),
 np.float32(6.1265335),
 np.float32(2.9294353),
 np.float32(1.1077064),
 np.float32(0.50608397),
 np.float32(0.32698762),
 np.float32(0.26281852),
 np.float32(0.22141452),
 np.float32(0.19525085),
 np.float32(0.18500362)]

In [410]:
# Simulation
V = 30
E = 0.25
VF = 1
VA = 1
VB = 2
CFLA = 1
CALA = 0
CFK = CFLA*2
CBK = 0
simulation_time = 81  # 시뮬레이션할 시간 스텝 수

input_values = [V, E, VF, VA, VB, CFLA, CALA, CFK, CBK]
normalized_inputs = normalize_inputs(input_values, model_range_mm)

# 초기 전류값 추정 (정규화된 0)
initial_I_norm = 0.0

# 초기 상태 텐서 생성 (10개 특성 모두 포함)
initial_state_values = normalized_inputs + [initial_I_norm]
initial_state_tensor = torch.tensor([initial_state_values]).float().to(device)

# Free Running 시뮬레이션 실행
with torch.no_grad():
    predictions = simulator_model(initial_state_tensor, simulation_time)

# 결과를 numpy로 변환
pred_normalized = predictions.cpu().numpy()  # [1, time_steps, 10]

# 비정규화하여 실제 물리적 값으로 변환
pred_physical = denormalize_outputs(pred_normalized, model_range_mm)

# 시간 배열 생성
time_steps = np.arange(simulation_time)

# 결과 데이터 저장
sim_res = {
    'time': time_steps,
    'V': pred_physical[0, :, 0],
    'E': pred_physical[0, :, 1], 
    'VF': pred_physical[0, :, 2],
    'VA': pred_physical[0, :, 3],
    'VB': pred_physical[0, :, 4],
    'CFLA': pred_physical[0, :, 5],
    'CALA': pred_physical[0, :, 6],
    'CFK': pred_physical[0, :, 7],
    'CBK': pred_physical[0, :, 8],
    'I': pred_physical[0, :, 9]
}

time = sim_res['time'] * 0.25 # hr
VF = sim_res['VF'] # L
VA = sim_res['VA'] # L
VB = sim_res['VB'] # L
CFLA = sim_res['CFLA'] # mol/L
CALA = sim_res['CALA'] # mol/L
CFK = sim_res['CFK'] # mol/L
CBK = sim_res['CBK'] # mol/L
I = sim_res['I'] # A
NALA = CALA *VA # mol
NBK = CBK *VB # mol
MwLA=90.1 #g/mol
Amem = 0.055 #m2
flux_mol = np.zeros_like(NALA) # mol/hr
flux_mol[0] = (NALA[1] - NALA[0]) / (time[1] - time[0])
flux_mol[1:-1] = (NALA[2:] - NALA[:-2]) / (time[2:] - time[:-2])
flux_mol[-1] = (NALA[-1] - NALA[-2]) / (time[-1] - time[-2])
flux_mol = flux_mol/3600 # mol/s
JLA = flux_mol*MwLA/Amem # g/m2/s
CE = 96485*flux_mol/10/I*100 # %
SEC = V*I/(MwLA*flux_mol*3600) # kwh/kg
SEC = np.where(SEC < 0, 0, SEC)

end_point = 0
for i in range(len(flux_mol)-1):
    if i>4 and flux_mol[i+1] <= 1e-8:
        end_point = i
        break
    else:
        end_point = i

time2 = time[:end_point]
VF2 = VF[:end_point]
VA2 = VA[:end_point]
VB2 = VB[:end_point]
CFLA2 = CFLA[:end_point]
CALA2 = CALA[:end_point]
CFK2 = CFK[:end_point]
CBK2 = CBK[:end_point]
I2 = I[:end_point]
NALA2 = NALA[:end_point]
NBK2 = NBK[:end_point]
CE2 = np.insert(CE[1:end_point], 0, 0)
JLA2 = JLA[:end_point]
SEC2 = SEC[:end_point]
res = {
    'time': time2,
    'VF': VF2,
    'VA': VA2,
    'VB': VB2,
    'CFLA': CFLA2,
    'CALA': CALA2,
    'CFK': CFK2,
    'CBK': CBK2,
    'I': I2,
    'CE': CE2,
    'NALA': NALA2,
    'NBK': NBK2,
    'JLA': JLA2,
    'SEC': SEC2,
}

# 1kg LA 분리 기준 회수율 및 에너지 소모량 분석 85wt% 농축 기준
NLA0 = res['CFLA'][0] * res['VF'][0] # mol
NK0 = res['CFK'][0] * res['VF'][0] # mol
target_Cm = 0.85 # kg/L = kg/kg

RR_LA = []
RR_K = []
E_BMED = []
E_PUMP = []
E_sensible = []
E_vapor = []
Total_E = []
Total_Time = []

i=10
outLA = res['NALA'][i] # mol
outK = res['NBK'][i] # mol

target_VA = outLA*MwLA/1000/target_Cm # L = kg

rr_LA = outLA/NLA0*100
rr_K = outK/NK0*100

dNLA = res['JLA'][i]*Amem*3600 # g/hr
Fv_out = dNLA/MwLA/res['CALA'][i] # L/hr (나가는 농도로 Acid volume flow rate 예상)
total_time = 1000/dNLA

e_BMED = V*res['I'][i]*total_time/1000 # kWh/kg
e_PUMP = 0.0047*total_time # kWh/kg
e_sensible = Fv_out*4.184*(100-20)*total_time/3600 # kWh/kg
e_vapor = (Fv_out - target_VA)*44/18*total_time/3600 # kWh/kg
Total_e = e_BMED + e_PUMP + e_sensible + e_vapor # kWh/kg

RR_LA.append(rr_LA)
RR_K.append(rr_K)
E_BMED.append(e_BMED)
E_PUMP.append(e_PUMP)
E_sensible.append(e_sensible)
E_vapor.append(e_vapor)
Total_E.append(Total_e)
Total_Time.append(total_time)




In [411]:
Fv_out

np.float32(0.9158907)

In [412]:
res['VA'][i]

np.float32(1.0029075)